Connecting to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tf.__version__

'2.12.0'

Importing Dataset

In [5]:
dataset = pd.read_csv('/content/drive/My Drive/483A10/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
Y = dataset.iloc[:, -1].values

In [6]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [7]:
print(Y)

[1 0 1 ... 1 1 0]


Encoding Categorical Data

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [9]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


Encoding Geography Column

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [11]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Splitting Dataset into training and testing

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

Feature Scaling

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Initializing the ANN

In [14]:
ann = tf.keras.models.Sequential()

Adding Input Layer and first hidden layer

In [15]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding second hidden layer

In [16]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding the output layer y dimension is 1, 

In [17]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Training the ANN

In [18]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Training ANN on training set batch size

In [19]:
ann.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.6197 - accuracy: 0.6846
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4762 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4576 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4462 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4351 - accuracy: 0.7980
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4243 - accuracy: 0.8018
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4122 - accuracy: 0.8081
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3991 - accuracy: 0.8144
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3872 - accuracy: 0.8185
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3774 - accura

Prediction

In [20]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 6000, 2, 1, 1, 5000]])) > 0.5)

1/1 [==============================] - 0s 104ms/step
[[False]]


Predicting Test set results

In [21]:
Y_pred = ann.predict(X_test)
Y_pred = (Y_pred > 0.5)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Making Confusion Matrix

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
accuracy_score(Y_test, Y_pred)

[[1509   86]
 [ 198  207]]


0.858